In [1]:
%run init.ipynb

C:\Users\Administrator\Anaconda3\lib\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.11) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


matchzoo version 1.0.2
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
停用表配置成功


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(
    truncated_length_left = 10,
    truncated_length_right = 100,
    filter_low_freq = 1
)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval:   0%| | 0/97 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 2.684 seconds.
Prefix dict has been built succesfully.
Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 97/97 [00:03<00:00, 24.34it/s]
Processing text_right with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 97/97 [00:01<00:00, 88.88it/s] 
Building Vocabulary from a datapack.: 100%|██████████████████████████████████████| 9

In [4]:
preprocessor.context

{'embedding_input_dim': 357,
 'filter_unit': <mzcn.preprocessors.units.frequency_filter.FrequencyFilter at 0x2ca311fec18>,
 'vocab_size': 357,
 'vocab_unit': <mzcn.preprocessors.units.vocabulary.Vocabulary at 0x2ca3ac77be0>}

In [5]:
# glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
# term_index = preprocessor.context['vocab_unit'].state['term_index']
# embedding_matrix = glove_embedding.build_matrix(term_index)
# l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
# embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
devset = mz.dataloader.Dataset(
    data_pack=dev_pack_processed
)

In [7]:
padding_callback = mz.models.DRMMTKS.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback,
)
devloader = mz.dataloader.DataLoader(
    dataset=devset,
    stage='dev',
    callback=padding_callback,
)

In [8]:
model = mz.models.DRMMTKS()

model.params['task'] = ranking_task
# model.params['embedding'] = embedding_matrix #这里是当加载glove等模型时取消该行注释
#设置embedding系数
model.params["embedding_output_dim"]=100
model.params["embedding_input_dim"]=preprocessor.context["embedding_input_dim"]
model.params['mask_value'] = 0
model.params['top_k'] = 10
model.params['mlp_activation_func'] = 'tanh'

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

DRMMTKS(
  (embedding): Embedding(357, 100, padding_idx=0)
  (attention): Attention(
    (linear): Linear(in_features=100, out_features=1, bias=False)
  )
  (mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=10, out_features=128, bias=True)
      (1): Tanh()
    )
    (1): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): Tanh()
    )
    (2): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): Tanh()
    )
    (3): Sequential(
      (0): Linear(in_features=128, out_features=1, bias=True)
      (1): Tanh()
    )
  )
  (out): Linear(in_features=1, out_features=1, bias=True)
)
Trainable params:  70363


In [9]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=devloader,
    validate_interval=None,
    epochs=10
)

In [10]:
trainer.run()

[Iter-1 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4062 - normalized_discounted_cumulative_gain@5(0.0): 0.4062 - mean_average_precision(0.0): 0.4062



[Iter-2 Loss-0.995]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4062 - normalized_discounted_cumulative_gain@5(0.0): 0.4062 - mean_average_precision(0.0): 0.4062



[Iter-3 Loss-0.988]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4062 - normalized_discounted_cumulative_gain@5(0.0): 0.4062 - mean_average_precision(0.0): 0.4062



[Iter-4 Loss-0.979]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4062 - normalized_discounted_cumulative_gain@5(0.0): 0.4062 - mean_average_precision(0.0): 0.4062



[Iter-5 Loss-0.963]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4062 - normalized_discounted_cumulative_gain@5(0.0): 0.4062 - mean_average_precision(0.0): 0.4062



[Iter-6 Loss-0.934]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4062 - normalized_discounted_cumulative_gain@5(0.0): 0.4062 - mean_average_precision(0.0): 0.4062



[Iter-7 Loss-0.877]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4062 - normalized_discounted_cumulative_gain@5(0.0): 0.4062 - mean_average_precision(0.0): 0.4062



[Iter-8 Loss-0.772]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4062 - normalized_discounted_cumulative_gain@5(0.0): 0.4062 - mean_average_precision(0.0): 0.4062



[Iter-9 Loss-0.616]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4062 - normalized_discounted_cumulative_gain@5(0.0): 0.4062 - mean_average_precision(0.0): 0.4062



[Iter-10 Loss-0.469]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4062 - normalized_discounted_cumulative_gain@5(0.0): 0.4062 - mean_average_precision(0.0): 0.4062

Cost time: 4.3155272006988525s


In [12]:
import gc
gc.collect()

0